# Challenge: Workshop and Challenge on Detection of Stress and Mental Health Using Wearable Sensors

<ul>
    <li><a href="#1">1. Data retrieval and cleaning</a></li>
</ul>
   
<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.1">1.1. Import libraries</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.2">1.2. Retrieve a sample of the SMILE dataset</a></li>
</ul>


<ul>
   <li><a href="#2">2. Data statistics</a></li>
</ul>

<ul>
   <li><a href="#3">3. Machine learning classifiers</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.1">3.1. Import libraries</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.2">3.2. Import predefined functions</a></li>
</ul>
<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.3">3.3. Classification</a></li>
</ul>

<a id="1"></a>

## 1. Data retrieval and cleaning

<a id="1.1"></a>
### 1.1. Import libraries

In [ ]:
pip install --upgrade pip

In [2]:
import pandas as pd
import numpy as np

<a id="1.2"></a>
### 1.2. Retrieve a sample of the SMILE dataset

The SMILE dataset was collected from 45 healthy adult participants (39 females and 6 males) in Belgium. The average age of participants was 24.5 years old, with a standard deviation of 3.0 years. Each participant contributed to an average of 8.7 days of data. Two types of wearable sensors were used for data collection. One was a wrist-worn device (Chillband, IMEC, Belgium) designed for the measurement of skin conductance (SC), ST, and acceleration data (ACC). The second sensor was a chest patch (Health Patch, IMEC, Belgium) to measure ECG and ACC. It contains a sensor node designed to monitor ECG at 256 Hz and ACC at 32 Hz continuously throughout the study period. Participants could remove the sensors while showering or before doing intense exercises. Also, participants received notifications on their mobile phones to report their momentary stress levels daily. 

https://compwell.rice.edu/workshops/embc2022/dataset

In [3]:
dataset = np.load('data/dataset_smile_challenge.npy', allow_pickle=True).item()
#     dict
#         dictionary with dataset, with keys:
#          * `train`
#           * `deep_features`
#            * `ECG_features_C`
#            * `ECG_features_T`
#            * `masking`
#           * `hand_crafted_features`
#            * `ECG_features`
#            * `ECG_masking`
#            * `GSR_features`
#            * `GSR_masking`
#           * `labels`
#          * `test`
#           * Same structure as `train`

Let's explore the contents of the dataset directory

<img src = "img/SMILE_dataset_features_3.png" width = "500" height = "400" >

[Souce: Click here](https://compwell.rice.edu/workshops/embc2022/challenge)

In [ ]:
# for training and testing data:
dataset_train = dataset['train']
dataset_test = dataset['test']
# for deep features.
deep_features = dataset_train['deep_features']
# conv1d backbone based features for ECG signal.
deep_features['ECG_features_C'] 
# transformer backbone base features for ECG signal  
deep_features['ECG_features_T']   
# for hand-crafted features.
handcrafted_features_train = dataset_train['hand_crafted_features']
handcrafted_features_test = dataset_test['hand_crafted_features']
# handcrafted features for ECG signal
handcrafted_features_train['ECG_features'] 
handcrafted_features_test['ECG_features']
# handcrafted features for GSR signal. 
handcrafted_features_train['GSR_features'] 
handcrafted_features_test['GSR_features']
# for labels.
labels_train = dataset_train['labels']  
labels_test = dataset_test['labels']

Now we have a DataFrame with the contents of the metadata file.

<a id="4"></a>
## 4. Data statistics 

In [4]:
print(
    f"train: {dataset['train']['hand_crafted_features']['ECG_features'].shape}")
print(
    f"test: {dataset['test']['hand_crafted_features']['ECG_features'].shape}")

train: (2070, 60, 8)
test: (986, 60, 8)


Load SMILE dataset as a dictionary from npy file.
Each feature matrix has 3 dimensions:
* sequence (of 60 minutes)
* window (5 minute with 4 min overlap)
* feature

#### ECG Features
* feature and label vector construction
* creation of classifier

In [ ]:
# represents the features extracted from one window of the sequence (corresponding to 5 minutes)
dataset['test']['hand_crafted_features']['ECG_features'][0][0]

In [ ]:
# Create an array with features
# Train dataset
nfeatures = len(dataset['train']['hand_crafted_features']['ECG_features'][0][0])
n = len(dataset['train']['hand_crafted_features']['ECG_features']) * \
    len(dataset['train']['hand_crafted_features']['ECG_features'])
# 
handcrafted_features_train=np.zeros((n,nfeatures)) # X
handcrafted_labels_train = np.zeros(n)  # y
#
count=0
for i in range(len(dataset['train']['hand_crafted_features']['ECG_features'])):
    for j in range(len(dataset['train']['hand_crafted_features']['ECG_features'][i])):
        if(np.sum(np.isnan(dataset['train']['hand_crafted_features']['ECG_features'][i][j])) == 0):
            # remove NAN
            handcrafted_features_train[count,
                                       0:nfeatures] = dataset['train']['hand_crafted_features']['ECG_features'][i][j]
            handcrafted_labels_train[count] = dataset['train']['labels'][i]
            count=count+1   
# Remove last values from the array
data_train = handcrafted_features_train
data_train[:-(n-count), :].shape
## Labels
data_train_label = handcrafted_labels_train
data_train_label[:-(n-count)].shape

In [ ]:
# Test dataset
nfeatures = len(dataset['test']['hand_crafted_features']['ECG_features'][0][0])
n = len(dataset['test']['hand_crafted_features']['ECG_features']) * \
    len(dataset['test']['hand_crafted_features']['ECG_features'][0])
# 
handcrafted_features_test=np.zeros((n,nfeatures)) #X
handcrafted_labels_test = np.zeros(n)  # y

count=0
for i in range(len(dataset['test']['hand_crafted_features']['ECG_features'])):
    #print(dataset['test']['hand_crafted_features']['ECG_features'][i])
    for j in range(len(dataset['test']['hand_crafted_features']['ECG_features'][i])):
        if(np.sum(np.isnan(dataset['test']['hand_crafted_features']['ECG_features'][i][j])) == 0):
            # remove NAN
            handcrafted_features_test[count,
                                      0:nfeatures] = dataset['test']['hand_crafted_features']['ECG_features'][i][j]
            handcrafted_labels_test[count] = dataset['test']['labels'][i]
            count = count+1
# Remove last values from the array
data_test = handcrafted_features_test
data_test[:-(n-count),:].shape  
## Labels
data_test_label = handcrafted_labels_test
data_test_label[:-(n-count)].shape

In [ ]:
## find max
print(np.where(data_train_label == np.max(data_train_label)))
## find min
print(np.where(data_train_label==np.min(data_train_label)))

In [ ]:
## find uniques
np.unique(data_train_label)

In [7]:
# Convert to a dataframe and save it in csv
df_train = pd.DataFrame(data=np.column_stack((data_train, data_train_label)))
df_train.columns = ["F"+str(i) for i in range(1, len(df_train.columns) + 1)]
df_train.rename(columns={'F9': 'Label'}, inplace=True)
df_train.head(n=10)

,F1,F2,F3,F4,F5,F6,F7,F8,Label
0,0.145656,0.152954,0.029353,0.013258,0.487958,0.272209,0.149786,0.056021,0.0
1,0.161642,0.037914,0.008152,0.015038,0.485591,0.273006,0.150057,0.061644,0.0
2,0.102252,0.007947,0.003004,0.026775,0.469134,0.222267,0.105493,0.101103,0.0
3,0.101629,0.007554,0.003805,0.035377,0.456785,0.069741,0.043349,0.124622,0.0
4,0.084450,0.012880,0.007234,0.039552,0.510779,0.250722,0.168897,0.120275,0.0
5,0.014152,0.062490,0.013785,0.015376,0.512783,0.264303,0.172324,0.024459,0.0
6,0.021252,0.121443,0.024277,0.013844,0.507665,0.265297,0.176635,0.018308,0.0
7,0.025347,0.076508,0.017720,0.016150,0.500325,0.267553,0.180807,0.023047,0.0
8,0.026834,0.017419,0.004964,0.020146,0.500671,0.261399,0.180753,0.032929,0.0
9,0.048197,0.010363,0.004642,0.031593,0.434076,0.107901,0.073289,0.055120,0.0


In [8]:
# Convert to a dataframe and save it in csv
df_test = pd.DataFrame(data=np.column_stack((data_test, data_test_label)))
df_test.columns = ["F"+str(i) for i in range(1, len(df_test.columns) + 1)]
df_test.rename(columns={'F9': 'Label'}, inplace=True)
df_test.head(n=10)


,F1,F2,F3,F4,F5,F6,F7,F8,Label
0,0.076467,0.095720,0.019321,0.010044,0.896764,0.117228,0.068013,0.103472,0.0
1,0.080711,0.110469,0.028849,0.014305,0.862462,0.151996,0.091932,0.105959,0.0
2,0.079685,0.075259,0.021964,0.016503,0.793803,0.142263,0.100381,0.115297,0.0
3,0.114616,0.047981,0.009772,0.010209,0.688007,0.128244,0.102531,0.061526,0.0
4,0.114824,0.018641,0.009672,0.032108,0.600439,0.106743,0.092132,0.073630,0.0
5,0.111699,0.007565,0.013751,0.114999,0.542728,0.096495,0.086296,0.152916,0.0
6,0.182996,0.007383,0.014673,0.125699,0.488070,0.016631,0.047455,0.331250,0.0
7,0.438657,0.007106,0.010648,0.093785,0.453511,0.019385,0.029299,0.503273,0.0
8,0.501287,0.006591,0.007833,0.073336,0.437255,0.019230,0.024552,0.375109,0.0
9,0.228813,0.008016,0.008132,0.063317,0.441105,0.057768,0.037015,0.199186,0.0


In [ ]:
df_train.to_csv('data/data_train.csv')
df_test.to_csv('data/data_test.csv')

<a id="3"></a>

## 3. Machine learning classifiers

<a id="3.1"></a>
### 3.1. Import libraries

In [9]:
import numpy.random.mtrand
from scipy.stats import pearsonr
from sklearn import model_selection, metrics, tree
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, \
    AdaBoostClassifier, IsolationForest
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import svm
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

<a id="3.2"></a>
### 3.2. Import predefined functions

In [10]:
from predefined_functions.logger import Logger, FeatureLogger

<a id="3.3"></a>
### 3.3. Classification

In [11]:
ds_train_handcrafted_features = dataset['train']['hand_crafted_features']
ds_train_labels = dataset['train']['labels']

ds_handcrafted_masking = dataset['train']['hand_crafted_features']['ECG_masking']
ds_deep_masking = dataset['train']['deep_features']['masking']

ds_test = dataset['test']

In [12]:
class FilteredDataset:

    def __init__(self, ecgFeatures, gsrFeatures, labels):
        self.ecgFeatures = ecgFeatures
        self.gsrFeatures = gsrFeatures
        self.labels = labels

#Returns an array with the arrays to keep in all major arrays (labels and features)

def filterZerosHandCraftedFeatures(handcraftedFeaturesDataset, handcraftedFeaturesLabels, filterZeros=True):

    #retornar array com indexes a manter ? (Masi facil)
    indexesToRemove = []

    for key in handcraftedFeaturesDataset.keys():
        if "masking" not in key:
            continue
        if not filterZeros:
            continue
        for i in range(len(handcraftedFeaturesDataset[key])):
            if np.min(handcraftedFeaturesDataset[key][i]) == 0:
                indexesToRemove.append(i)

    indexesToKeep = range(len(handcraftedFeaturesLabels))
    #else: indexesToKeep = np.delete(range(len(handcraftedFeaturesLabels)), np.unique(indexesToRemove))

    return FilteredDataset(ecgFeatures=handcraftedFeaturesDataset['ECG_features'][indexesToKeep],
                           gsrFeatures=handcraftedFeaturesDataset['GSR_features'][indexesToKeep],
                           labels=handcraftedFeaturesLabels[indexesToKeep])


def discretizeHandCraftedDataset(processedHandTrainDataset):
    
    n_entries = len(processedHandTrainDataset.labels)
    nOfFeatures = 5
    dataset_features = np.zeros((n_entries, nOfFeatures))
    #Array positions
    # 0 - mean minute hr
    # 0 - mean gsr

    for index_n_entries in range(n_entries):

        dataset_features[index_n_entries] = [

            ####### HR Features
            # 0.20 , 0.006
            np.nanmean(
                [i[0] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]]),
            # np.nanpercentile([i[1] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]], q=80),
            # np.nanquantile([i[2] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]],q=0.4),
            # np.nansum([i[3] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]]),
            np.nanmean(
                [i[7] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]]),
            # np.nanquantile([i[5] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]], q=0.90),
            ## 0.156 , 0.001
            # np.nanpercentile([i[4] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]], q=85), ## 0.148 , 0.014
            # np.nanpercentile([i[6] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]], q=10), #0.145, covariance :0.003
            # np.nansum([i[4] for i in processedHandTrainDataset.ecgFeatures[index_n_entries]]), # 0.153 , 0.938

            ######## GSR Features
            np.nanvar(
                [i[4] for i in processedHandTrainDataset.gsrFeatures[index_n_entries]]),
            np.nanquantile([i[5] for i in processedHandTrainDataset.gsrFeatures[index_n_entries]],
                           q=0.85) - np.nanquantile(
                [i[0] for i in processedHandTrainDataset.gsrFeatures[index_n_entries]], q=0.15),
            # -0.009, covariance :-0.000
            # np.nanvar([i[4] for i in processedHandTrainDataset.gsrFeatures[index_n_entries]]),
            # -0.009, covariance :-0.000
            ######## ST Features
            np.nanquantile(
                [i[10] for i in processedHandTrainDataset.gsrFeatures[index_n_entries]], q=0.95),
        ]

    return dataset_features


def experimentClassifier(clf, dataset, datasetLabels, nrOfIters=1, binarizeOutputLabels=False, binarizeAllLabels=False):
    sgkf = model_selection.StratifiedKFold(n_splits=3, shuffle=False)

    for trainIndexes, testIndexes in sgkf.split(dataset, datasetLabels):
        kfold_train_dataset = dataset[trainIndexes]
        kfold_train_dataset_labels = datasetLabels[trainIndexes]
        kfold_test_dataset = dataset[testIndexes]
        kfold_test_dataset_labels = datasetLabels[testIndexes]

        if binarizeAllLabels:
            kfold_train_dataset_labels = np.where(
                kfold_train_dataset_labels == 0, 0, 1)
            kfold_test_dataset_labels = np.where(
                kfold_test_dataset_labels == 0, 0, 1)

        clf.fit(kfold_train_dataset, kfold_train_dataset_labels)
        kfold_predicted_labels = clf.predict(kfold_test_dataset)

        #Printing results
        if binarizeOutputLabels or binarizeAllLabels:
            Logger.logPredictionMetrics(np.where(
                kfold_test_dataset_labels == 0, 0, 1), np.where(kfold_predicted_labels == 0, 0, 1))
        else:
            Logger.logPredictionMetrics(
                kfold_test_dataset_labels, kfold_predicted_labels)

        nrOfIters = nrOfIters-1
        if nrOfIters == 0:
            break

def fitAndPredictClassifier(clf, trainDataset, trainDatasetLabels, testDataset, testDatasetLabels=None, binarizeLabels=True, printMetrics=False, saveOutputFile=True):
    sgkf = model_selection.StratifiedKFold(n_splits=2, shuffle=False,)

    #clf.fit(trainDataset, trainDatasetLabels)
    predicted_labels = clf.predict(testDataset)

    if printMetrics:
        Logger.logPredictionMetrics(testDatasetLabels, predicted_labels)

    if binarizeLabels:
        predicted_labels = np.where(predicted_labels == 0, 0, 1)

    if saveOutputFile:
        np.savetxt('/data/answer.txt', predicted_labels, fmt='%i')

def reg_m(x, y):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, x).fit()
    return results

def balanced_subsample(x, y, subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci, this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs, ys

In [ ]:
# --------------  PROCESS DATA  -------------- #

processedHandTrainDataset = filterZerosHandCraftedFeatures(
    ds_train_handcrafted_features, ds_train_labels)

trainDatasetTest = discretizeHandCraftedDataset(processedHandTrainDataset)

startTrainIndex = 0
endTrainIndex = len(trainDatasetTest)

trainDataset = trainDatasetTest[startTrainIndex:endTrainIndex]
trainDatasetLabels = np.where(processedHandTrainDataset.labels[startTrainIndex:endTrainIndex]==0,0,1)

trainDataset, trainDatasetLabels= balanced_subsample(trainDataset,trainDatasetLabels)

In [ ]:
# define the model
importanceModel = DecisionTreeRegressor()
# fit the model
importanceModel.fit(trainDataset, trainDatasetLabels)
# get importance
importance = importanceModel.feature_importances_
importance=np.sort(importance)
# summarize feature importance
for i,v in enumerate(importance):
    #Useful to remove features that wont be used for the model
    print('Feature: %0d, Importance Score: %.5f' % (i, v))

#for featureIndex in range(len(trainDataset[0])) :
#corr, _ = pearsonr(dataset_handcrafted_masking.flatten() , dataset_deep_masking.flatten())
#covariance= np.cov(dataset_handcrafted_masking.flatten(), dataset_deep_masking.flatten())[0][1]
#print('Feature: %0d, Pearsons correlation: %.3f, covariance :%.3f' % (0, corr,covariance))

#for featureIndex in range(len(trainDataset[0])) :
#    corr, _ = pearsonr([i[featureIndex] for i in trainDataset], binarizedTrainLabels)
#    covariance= np.cov([i[featureIndex] for i in trainDataset], binarizedTrainLabels)[0][1]
#    print('Feature: %0d, Pearsons correlation: %.3f, covariance :%.3f' % (featureIndex, corr,covariance))
#print(reg_m(trainDataset, trainDatasetLabels).summary())

FeatureLogger.logFeatureRankings(trainDataset,trainDatasetLabels)
FeatureLogger.logFeaturesCorrelation(trainDataset,trainDatasetLabels)

#exit(0)
#clf = tree.DecisionTreeClassifier(class_weight=[{0: 1, 1: 5}, {0: 1, 1: 5}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 0}])
#clf = tree.DecisionTreeClassifier(class_weight={0: 2, 1: 1, 2: 2, 3: 3}) #0.54
#clf = RandomForestClassifier(max_depth=18,max_features='log2')
#clf = RandomForestClassifier(min_samples_leaf=10,max_depth=10, max_features=None) #BEST CLASSIFIER SO FAR - 0.45
clf = svm.LinearSVC(class_weight={0:1.2,1:1,3:0,4:1}) # 0.53

experimentClassifier(clf, trainDataset, trainDatasetLabels,nrOfIters=3, binarizeAllLabels=True)

In [ ]:
# --------------  TESTING  -------------- #

ds_test_handcrafted_features = dataset['test']['hand_crafted_features']
ds_test_labels = dataset['test']['labels']
processedHandTestDataset=filterZerosHandCraftedFeatures(ds_test_handcrafted_features,ds_test_labels,filterZeros=False)

testDatasetTest = discretizeHandCraftedDataset(processedHandTestDataset)

fitAndPredictClassifier(clf, trainDataset, trainDatasetLabels, testDatasetTest)